In [1]:
import os
import pandas as pd

In [2]:
TRI_DATA_DIR = './trip_data/'
CLI_DATA_DIR = './climate_data/'

## Data Loading 

In [3]:
df_check = pd.read_csv(TRI_DATA_DIR+'stations_check.csv')

In [4]:
df_check.head()

,Station Id,Station Name,Year,Month,Day,Time,Weekend,Holiday,Checkin per Hour,Checkout per Hour
0,7021,Bay St / Albert St,2019,1,1,00:00,0,1,0.0,1.0
1,7021,Bay St / Albert St,2019,1,1,01:00,0,1,0.0,0.0
2,7021,Bay St / Albert St,2019,1,1,02:00,0,1,0.0,0.0
3,7021,Bay St / Albert St,2019,1,1,03:00,0,1,0.0,0.0
4,7021,Bay St / Albert St,2019,1,1,04:00,0,1,0.0,0.0


In [5]:
df_cluster = pd.read_csv(TRI_DATA_DIR+'stations_classified_BGMM.csv')

In [6]:
df_cluster.head()

,id,name,lat,lon,cluster_1,cluster_2,prev_cluster
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954,1,0,5
1,7001,Lower Jarvis St / The Esplanade,43.647830,-79.370698,26,6,29
2,7002,St. George St / Bloor St W,43.667333,-79.399429,12,2,22
3,7003,Madison Ave / Bloor St W,43.667158,-79.402761,13,2,18
4,7004,University Ave / Elm St,43.656518,-79.389099,6,0,7


In [7]:
df_climate = pd.read_csv(CLI_DATA_DIR+'climate.csv')

In [8]:
df_climate.head()

,Climate ID,Date/Time,Year,Month,Day,Time,Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Spd (km/h),...,Freezing Rain,Haze,Heavy Rain,Heavy Snow,Moderate Rain,Moderate Snow,Rain,Snow,Thunderstorms,Snow on Grnd (cm)
0,6158359,2019-01-01 00:00,2019,1,1,00:00,4.3,4.0,98.0,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,6158359,2019-01-01 01:00,2019,1,1,01:00,5.1,5.0,99.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6158359,2019-01-01 02:00,2019,1,1,02:00,5.9,5.6,98.0,28.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,6158359,2019-01-01 03:00,2019,1,1,03:00,3.3,2.6,95.0,34.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6158359,2019-01-01 04:00,2019,1,1,04:00,2.8,1.2,89.0,28.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Cluster ID

In [9]:
df_cluster.rename(columns={'id':'Station Id', 'cluster_1':'Cluster'}, inplace=True)

In [10]:
df_cluster = df_cluster[['Station Id', 'Cluster']]

In [11]:
df = df_check.merge(df_cluster, on='Station Id', how='left')
df.head()

,Station Id,Station Name,Year,Month,Day,Time,Weekend,Holiday,Checkin per Hour,Checkout per Hour,Cluster
0,7021,Bay St / Albert St,2019,1,1,00:00,0,1,0.0,1.0,22
1,7021,Bay St / Albert St,2019,1,1,01:00,0,1,0.0,0.0,22
2,7021,Bay St / Albert St,2019,1,1,02:00,0,1,0.0,0.0,22
3,7021,Bay St / Albert St,2019,1,1,03:00,0,1,0.0,0.0,22
4,7021,Bay St / Albert St,2019,1,1,04:00,0,1,0.0,0.0,22


In [12]:
# groupby cluster
df = df.groupby(['Cluster', 'Year', 'Month', 'Day', 'Time']).sum().reset_index()
df['Weekend'] = df['Weekend'] > 0
df['Weekend'] = df['Weekend'].astype(int)
df['Holiday'] = df['Holiday'] > 0
df['Holiday'] = df['Holiday'].astype(int)
df.drop(columns=['Station Id'], inplace=True)
df.head()

,Cluster,Year,Month,Day,Time,Weekend,Holiday,Checkin per Hour,Checkout per Hour
0,0,2019,1,1,00:00,0,1,0.0,1.0
1,0,2019,1,1,01:00,0,1,4.0,4.0
2,0,2019,1,1,02:00,0,1,1.0,4.0
3,0,2019,1,1,03:00,0,1,4.0,1.0
4,0,2019,1,1,04:00,0,1,2.0,0.0


## Add Climate Data

In [13]:
df_climate.drop(columns=['Climate ID', 'Date/Time'], inplace=True)

In [14]:
df = df.merge(df_climate, on=['Year', 'Month', 'Day', 'Time'], how='left')

In [15]:
df.head()

,Cluster,Year,Month,Day,Time,Weekend,Holiday,Checkin per Hour,Checkout per Hour,Temp (°C),...,Freezing Rain,Haze,Heavy Rain,Heavy Snow,Moderate Rain,Moderate Snow,Rain,Snow,Thunderstorms,Snow on Grnd (cm)
0,0,2019,1,1,00:00,0,1,0.0,1.0,4.3,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0,2019,1,1,01:00,0,1,4.0,4.0,5.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2019,1,1,02:00,0,1,1.0,4.0,5.9,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0,2019,1,1,03:00,0,1,4.0,1.0,3.3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,2019,1,1,04:00,0,1,2.0,0.0,2.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
df.groupby('Cluster').count()['Year']

Cluster
0     8760
1     8760
2     8760
3     8760
4     8760
5     8760
6     8760
7     8760
8     8760
9     8760
10    8760
11    8760
12    8760
13    8760
14    8760
15    8760
16    8760
17    8760
18    8760
19    8760
20    8760
21    8760
22    8760
23    8760
24    8760
25    8760
26    8760
27    8760
28    8760
29    8760
Name: Year, dtype: int64

## Save to CSV 

In [17]:
df.to_csv('./trips_BGMM.csv', index=False)